In [141]:
# UPDATE TO MATCH PERSONAL DIRECTORY STRUCTURE
DIR = 'C:\\Users\\jesse\\Documents\\GitHub\\vanity_sizing_project\\data\\'

In [142]:
import pandas as pd
import pickle

In [143]:
with open(DIR+'rtr_reviews_uncleaned.pkl', 'rb') as f:
    data = pickle.load(f)

customers = data[['user_id', 'weight', 'body type', 'height', 'size', 'age','band_size', 'cup_size']]

customers = customers.drop_duplicates(subset='user_id')

In [144]:
customers.isnull().sum()

user_id          0
weight       18271
body type     9447
height         466
size             0
age            636
band_size    11391
cup_size     11391
dtype: int64

In [145]:
# Get the list of non-numerical columns
non_numerical_cols = customers.select_dtypes(exclude=['int64', 'float64']).columns

# Apply ordinal encoding to all non-numerical columns
for col in non_numerical_cols:
    customers[col] = pd.factorize(customers[col])[0]

In [146]:
customer_nulls = customers[customers.isnull().any(axis=1)]
customers_complete = customers.dropna()

In [147]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from math import sqrt
import numpy as np
import pandas as pd

def predict_fill_regress(complete_df, col):
    X = complete_df.drop(columns=[col, 'user_id'])
    y = complete_df[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(sqrt(mean_squared_error(y_test, y_pred)))
    return model

def predict_fill_classify(complete_df, original_col):
    # Reverse the one-hot encoding
    X = complete_df.drop(columns=[original_col, 'user_id'])
    y = complete_df[original_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    return model

In [148]:
predict_fill_regress(customers_complete,'height')

2.2870413106633465


RandomForestRegressor()

In [149]:
predict_fill_regress(customers_complete,'weight')

11.13431941839157


RandomForestRegressor()

In [150]:
#for body type, age, band size, and cup size, use a classifier
predict_fill_classify(customers_complete,'body type')

0.3608116677235257


RandomForestClassifier()

In [151]:
predict_fill_regress(customers_complete,'age')

8.532749011283341


RandomForestRegressor()

In [152]:
predict_fill_classify(customers_complete,'band_size')

0.5868449876059261


RandomForestClassifier()

In [155]:
predict_fill_classify(customers_complete,'cup_size')

0.3965527180492304


RandomForestClassifier()

In [154]:
from sklearn.ensemble import GradientBoostingClassifier

def predict_fill_classify_boosted(complete_df, original_col):
    # Reverse the one-hot encoding
    X = complete_df.drop(columns=[original_col, 'user_id'])
    y = complete_df[original_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Use GradientBoostingClassifier instead of RandomForestClassifier
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    return model

In [156]:
predict_fill_classify_boosted(customers_complete,'body type')

0.4443419611460195


GradientBoostingClassifier()